# Introduction to plotly

<table>
<tr>
<td><img src="https://i.imgur.com/BqJgyzB.png" width="350px"/></td>
<td><img src="https://i.imgur.com/ttYzMwD.png" width="350px"/></td>
<td><img src="https://i.imgur.com/WLmzj41.png" width="350px"/></td>
<td><img src="https://i.imgur.com/LjRTbCn.png" width="350px"/></td>
</tr>
<tr>
<td style="font-weight:bold; font-size:16px;">Scatter Plot</td>
<td style="font-weight:bold; font-size:16px;">Choropleth</td>
<td style="font-weight:bold; font-size:16px;">Heatmap</td>
<td style="font-weight:bold; font-size:16px;">Surface Plot</td>
</tr>
<tr>
<td>go.Scatter()</td>
<td>go.Choropleth()</td>
<td>go.Heatmap()</td>
<td>go.Surface()</td>
</tr>
<!--
<tr>
<td>Good for interval and some nominal categorical data.</td>
<td>Good for interval and some nominal categorical data.</td>
<td>Good for nominal and ordinal categorical data.</td>
<td>Good for ordinal categorical and interval data.</td>
</tr>
-->
</table>


到目前为止，在本教程中我们一直在使用`seaborn`和`pandas`，两个成熟的库围绕`matplotlib`设计。 这些库都专注于构建“静态”可视化：没有移动部件的可视化。 换句话说，到目前为止我们构建的所有图都可以出现在死树期刊文章中。

在交互性和动画方面，网络解锁了很多可能性。 有许多绘图库可供尝试提供这些功能。

在本节中，我们将研究`plotly`，这是一个开源绘图库，是这些库中最受欢迎的库之一。

In [3]:
import pandas as pd
reviews = pd.read_csv("./input/winemag-data-130k-v2.csv", index_col=0)
reviews.head()

country                                        description  \
0     Italy  Aromas include tropical fruit, broom, brimston...   
1  Portugal  This is ripe and fruity, a wine that is smooth...   
2        US  Tart and snappy, the flavors of lime flesh and...   
3        US  Pineapple rind, lemon pith and orange blossom ...   
4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  \
0                 Etna                NaN       Kerin O’Keefe   
1                  NaN                NaN          Roger Voss   
2    Willamette Valley  Willamette Valley        Paul Gregutt   
3  Lake Michigan Shore                NaN  Alexander Peartree   
4    Willamette Valley  Willamette Valley        Paul Gregutt   

  taster_twitter_handle                                              title  \
0          @kerinokeefe                  Nicosia 2013 Vulkà Bianco  (Etna)   
1            @vossroger      Quinta dos Avidagos 2011 Avidagos Red (Douro)   
2           @paulgwine       Rainstorm 2013 Pinot Gris (Willamette Valley)   
3                   NaN  St. Julian 2013 Reserve Late Harvest Riesling ...   
4           @paulgwine   Sweet Cheeks 2012 Vintner's Reserve Wild Child...   

          variety               winery  
0     White Blend              Nicosia  
1  Portuguese Red  Quinta dos Avidagos  
2      Pinot Gris            Rainstorm  
3        Riesling           St. Julian  
4      Pinot Noir         Sweet Cheeks

`plotly`提供在线和离线模式。 后者将`plotly`源代码直接注入笔记本中; 前者没有。 我建议绝大多数时候在离线模式下使用`plotly`，这是唯一适用于Kaggle的模式（在Python中禁用网络访问）。

In [5]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

我们首先创建一个基本的散点图。

In [14]:
import plotly.graph_objs as go

iplot([go.Scatter(x=reviews.head(1000)['points'], y=reviews.head(1000)['price'], mode='markers')])

此图表完全是交互式的。 我们可以使用右上角的工具栏对数据执行各种操作：例如缩放和平移。 当我们将鼠标悬停在数据点上时，我们会得到一个工具提示。 我们甚至可以将绘图保存为PNG图像。

此图表还显示了这个更高级绘图库的*缺点*。 为了保持合理的性能，我们不得不将自己限制在数据集中的前1000个点。 虽然这是必要的（为了避免过多的过度绘图），但重要的是要注意，一般来说，交互式图形比静态图形更加资源密集。 更容易“最大化”您可以显示的数据点数。

注意`plotly` API的“形状”。 `iplot`获取一个绘图对象列表并为你编写它们，绘制组合的最终结果。 这样可以轻松堆叠图。

另一个例子，这是一个KDE图（什么'plotly`指的是`Histogram2dContour`）*和*相同数据的散点图。

In [7]:
iplot([go.Histogram2dContour(x=reviews.head(500)['points'], 
                             y=reviews.head(500)['price'], 
                             contours=go.Contours(coloring='heatmap')),
       go.Scatter(x=reviews.head(1000)['points'], y=reviews.head(1000)['price'], mode='markers')])

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/plotly/graph_objs/_deprecations.py:204: DeprecationWarning:

plotly.graph_objs.Contours is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.contour.Contours
  - plotly.graph_objs.surface.Contours
  - etc.




`plotly`暴露了几种不同的API，范围从低级到高级。 `iplot`是最高级别的API，因此是最方便的通用API。

就我个人而言，我总是发现`plotly``Surface`是最令人印象深刻的功能（虽然是最难做到的一个）：

In [8]:
df = reviews.assign(n=0).groupby(['points', 'price'])['n'].count().reset_index()
df = df[df["price"] < 100]
v = df.pivot(index='price', columns='points', values='n').fillna(0).values.tolist()

In [9]:
iplot([go.Surface(z=v)])

在Kaggle，'plotly`经常被用来制作**choropleths**。 等值线是一种地图，其中所有实体（国家，美国州等）根据数据集中的某个变量着色。 `plotly`是可用于制作它们的最方便的绘图库之一。

In [10]:
df = reviews['country'].replace("US", "United States").value_counts()

iplot([go.Choropleth(
    locationmode='country names',
    locations=df.index.values,
    text=df.index,
    z=df.values
)])

总的来说，`plotly`是一个功能强大，交互式丰富的数据可视化库。 它允许我们生成比标准的“pandas”或“seaborn”输出更多“pizazz”的情节。

权衡的是，虽然`pandas`和`seaborn`已经很成熟，但'plotly`仍然是新的。 因此，特别是“情节”文档更难以找到和解释; `plotly`网站上的办公文档使用了不同功能的混合进行绘图，这使得它比以前更难使用。

此外，重要的是要识别交互性何时有用，何时不是。 最有效的图不需要使用悬停或工具提示来完成他们的工作。 因此，“情节”虽然非常有吸引力，但却比“熊猫”或“seaborn”中的同等情节更有用*。

# Exercises

对于以下练习，请尝试分叉并运行此笔记本，然后再现下面的图表。 提示：x轴上的“攻击”，y轴上的“防御”。

In [12]:
import pandas as pd
pokemon = pd.read_csv("./input/Pokemon1.csv")
pokemon.head(3)

#       Name Type 1  Type 2  Total  HP  Attack  Defense  Sp. Atk  Sp. Def  \
0  1  Bulbasaur  Grass  Poison    318  45      49       49       65       65   
1  2    Ivysaur  Grass  Poison    405  60      62       63       80       80   
2  3   Venusaur  Grass  Poison    525  80      82       83      100      100   

   Speed  Generation  Legendary  
0     45           1      False  
1     60           1      False  
2     80           1      False

In [13]:
import plotly.graph_objs as go

iplot([go.Scatter(x=pokemon['Attack'], y=pokemon['Defense'], mode='markers')])

# Conclusion

在本节中，我们查看了`plotly`，这是一个交互式绘图库，可生成非常吸引人的图表。 它是基于“matplotlib”的工具的众多替代品之一，提供一流的交互性（`bokeh`是另一个值得一提的）。

在下一节中，我们将看另一个绘图库，`plotnine`，它是围绕一种奇特但强大的想法设计的，称为**图形语法**。